In [1]:
import warnings
warnings.filterwarnings("ignore")
from dataset import AudioDataset
from torch.utils.data import DataLoader
import torch.optim as optim
from torch import nn
import torch
from model import LSTMnet_RnnAtten, LSTMnet_GateAtten
import os 
os.chdir('../')

In [2]:

print(os.getcwd())

train_annotations = '.\\data\\MELD\\train_sent_emo.csv'
val_annotations = 'data\MELD\dev_sent_emo.csv'

train_audio = '.\\data\\MELD\\audio\\wav\\'
val_audio = '.\\data\\MELD\\audio\\wav_dev\\'


c:\Users\kjart\OneDrive\Dokumenter\KU\2. semester\CogSci2\research\multimodal


In [3]:
train = AudioDataset(train_annotations, train_audio)
val = AudioDataset(val_annotations, val_audio)

train_dataloader = DataLoader(train, batch_size=32, shuffle = True, drop_last=True)
val_dataloader = DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
train_features, train_labels = next(iter(train_dataloader))

In [4]:
len(train)

9988

In [5]:
# Sanity check: Size should be [batch_size, num_features, feature_vector_len], [batch_size]
# Where batch size is 32, num_features (num mfccs) is 40 
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
features = train_features[0]
label = train_labels[0]
features,label


Feature batch shape: torch.Size([32, 1, 100])
Labels batch shape: torch.Size([32])


(tensor([[-2.0008e+01,  3.6507e+01, -8.3521e+00,  2.4488e+00, -3.5428e+00,
           5.9425e-03, -6.4703e+00, -2.7236e+00,  1.7968e+00,  1.7713e+00,
          -3.1142e+00, -1.4320e+00,  6.3949e-01, -5.6315e-01, -4.0826e+00,
          -3.0969e+00, -2.3699e+00, -1.0014e+00, -1.6065e+00, -2.1289e+00,
          -2.2644e+00,  4.7880e-01, -8.7934e-01,  5.6708e-01, -8.1863e-01,
           8.6900e-01, -5.0186e-01, -2.9704e-01, -3.3655e-01, -1.5720e+00,
          -1.7167e+00, -1.4203e+00, -4.2611e-01,  1.7261e+00,  9.2214e-01,
           3.2988e+00,  1.1587e+00, -1.8606e-01, -1.1648e+00, -1.9743e+00,
          -1.1160e+00, -8.6839e-01, -6.4197e-01, -1.1396e-01, -9.6240e-01,
           7.2914e-01, -5.4036e-01, -1.0283e+00,  1.3328e-01, -9.6430e-01,
          -6.8130e-01, -1.5646e-01, -1.0815e+00, -4.1762e-02, -1.2825e+00,
          -4.5615e-02, -5.5349e-01, -3.9800e-01, -4.7965e-01, -6.1431e-01,
           2.6892e-01, -1.4437e-02, -9.8577e-01, -1.3096e+00, -9.3351e-01,
          -4.3267e-01, -5

In [6]:
for batch in train_dataloader:
    x, y = batch
    print(x.shape, y.shape)
    break

torch.Size([32, 1, 100]) torch.Size([32])


In [7]:
def loss_fnc(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions,target=targets)

In [8]:
def make_train_step(model, loss_fnc, optimizer):
    def train_step(X,Y):
        # set model to train mode
        model.train()
        # forward pass
        output_logits = model(X)
        predictions = torch.argmax(output_logits,dim=1)
        accuracy = torch.sum(Y==predictions)/float(len(Y))
        # compute loss
        loss = loss_fnc(output_logits, Y)
        # compute gradients
        loss.backward()
        # update parameters and zero gradients
        optimizer.step()
        optimizer.zero_grad()
        return loss.item(), accuracy*100
    return train_step

In [9]:
def make_validate_fnc(model,loss_fnc):
    def validate(X,Y):
        with torch.no_grad():
            model.eval()
            output_logits = model(X)
            predictions = torch.argmax(output_logits,dim=1)
            accuracy = torch.sum(Y==predictions)/float(len(Y))
            loss = loss_fnc(output_logits,Y)
        return loss.item(), accuracy*100, predictions
    return validate

In [18]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model = LSTMnet_RnnAtten(input_dim = 100, hidden_dim=32, output_dim=7, num_layers=3).to(device)
print('Number of trainable params: ',sum(p.numel() for p in model.parameters()) )
OPTIMIZER = torch.optim.Adam(model.parameters(),lr=0.0001, weight_decay=1e-3)


cuda:0
Number of trainable params:  40583


TypeError: __init__() got an unexpected keyword argument 'device'

In [17]:
train_step = make_train_step(model, loss_fnc, optimizer=OPTIMIZER)
validate = make_validate_fnc(model,loss_fnc)

losses=[]
val_losses = []
epochs = 30
for epoch in range(epochs):
    epoch_acc = 0
    epoch_loss = 0
    for idx, (features, labels) in enumerate(train_dataloader):
        X, Y = features, labels
        X_tensor = torch.tensor(X,device=device).float()
        Y_tensor = torch.tensor(Y, dtype=torch.long,device=device)
        loss, acc = train_step(X_tensor,Y_tensor)
        epoch_acc += acc*len(features)/len(train)
        epoch_loss += loss*len(features)/len(train)
        print(f"\r Epoch {epoch}: batch {idx}",end='')
    for batch in val_dataloader:
        X_val, Y_val = batch
        break
    X_val_tensor = torch.tensor(X_val,device=device).float()
    Y_val_tensor = torch.tensor(Y_val,dtype=torch.long,device=device)
    val_loss, val_acc, _ = validate(X_val_tensor,Y_val_tensor)
    losses.append(epoch_loss)
    val_losses.append(val_loss)
    print('')
    print(f"Epoch {epoch} --> loss:{epoch_loss:.4f}, acc:{epoch_acc:.2f}%, val_loss:{val_loss:.4f}, val_acc:{val_acc:.2f}%")

 Epoch 0: batch 81

KeyboardInterrupt: 

In [ ]:
#torch.save(model.state_dict(), 'audio/LSTMnet_RnnAtten_weights')